Perguntas:

* Quais UFs com maiores saldo devedor?
* Quais UFs com maiores valores de empréstimo?
* Quais setores com maiorer saldo devedor?
* Quais setores com maiores valores de empréstimo?
* Qual a média de pd por setor?
* Qual a média de score por setor?
* Qual a média de pe por setor?
* Qual canal mais utilizado em cada mes e uf no último ano?
* Qual a média de pd por idade?
* Qual a média de score por idade?
* Qual a média de pe por idade?

## Geografia

In [0]:
WITH ult_mes AS (
  SELECT max(dt_mes) AS dt_mes
  FROM puc.gold.fato_risco_cliente_mes
)
SELECT
  l.uf,
  SUM(f.ead_soma) AS ead_total
FROM puc.gold.fato_risco_cliente_mes f
JOIN puc.gold.dim_localizacao l
  ON l.sk_localizacao = f.sk_localizacao
JOIN ult_mes u
  ON f.dt_mes = u.dt_mes
GROUP BY l.uf
ORDER BY ead_total DESC;

In [0]:
WITH ult_mes AS (
  SELECT max(dt_mes) AS dt_mes
  FROM puc.gold.fato_risco_cliente_mes
)
SELECT
  l.regiao,
  SUM(f.ead_soma) AS ead_total
FROM puc.gold.fato_risco_cliente_mes f
JOIN puc.gold.dim_localizacao l
  ON l.sk_localizacao = f.sk_localizacao
JOIN ult_mes u
  ON f.dt_mes = u.dt_mes
GROUP BY l.regiao
ORDER BY ead_total DESC;

In [0]:
WITH ult_mes AS (
  SELECT max(dt_mes) AS dt_mes
  FROM puc.gold.fato_risco_cliente_mes
)
SELECT
  l.uf,
  AVG(f.pd_model_media) AS pd_model_media_simples,
  SUM(f.pd_model_media * f.ead_soma) / NULLIF(SUM(f.ead_soma), 0) AS pd_model_media_pond_ead,
  SUM(f.ead_soma) AS ead_total
FROM puc.gold.fato_risco_cliente_mes f
JOIN puc.gold.dim_localizacao l
  ON l.sk_localizacao = f.sk_localizacao
JOIN ult_mes u
  ON f.dt_mes = u.dt_mes
GROUP BY l.uf
ORDER BY pd_model_media_pond_ead DESC;


In [0]:
WITH ult_mes AS (
  SELECT max(dt_mes) AS dt_mes
  FROM puc.gold.fato_risco_cliente_mes
)
SELECT
  l.uf,
  AVG(f.pd_true_media) AS pd_true_media_simples,
  SUM(f.pd_true_media * f.ead_soma) / NULLIF(SUM(f.ead_soma), 0) AS pd_true_media_pond_ead,
  SUM(f.ead_soma) AS ead_total
FROM puc.gold.fato_risco_cliente_mes f
JOIN puc.gold.dim_localizacao l
  ON l.sk_localizacao = f.sk_localizacao
JOIN ult_mes u
  ON f.dt_mes = u.dt_mes
GROUP BY l.uf
ORDER BY pd_true_media_pond_ead DESC;


In [0]:
WITH ult_mes AS (
  SELECT max(dt_mes) AS dt_mes
  FROM puc.gold.fato_risco_cliente_mes
),
agg AS (
  SELECT
    l.uf,
    SUM(f.ead_soma) AS ead_total,
    SUM(f.pd_model_media * f.ead_soma) / NULLIF(SUM(f.ead_soma), 0) AS pd_model_pond,
    SUM(f.pd_true_media  * f.ead_soma) / NULLIF(SUM(f.ead_soma), 0) AS pd_true_pond
  FROM puc.gold.fato_risco_cliente_mes f
  JOIN puc.gold.dim_localizacao l
    ON l.sk_localizacao = f.sk_localizacao
  JOIN ult_mes u
    ON f.dt_mes = u.dt_mes
  GROUP BY l.uf
)
SELECT
  uf,
  ead_total,
  pd_model_pond,
  pd_true_pond,
  (pd_true_pond - pd_model_pond) AS gap_pd_true_menos_model
FROM agg
ORDER BY gap_pd_true_menos_model DESC;


In [0]:
WITH ult_mes AS (
  SELECT max(dt_mes) AS dt_mes
  FROM puc.gold.fato_risco_cliente_mes
)
SELECT
  l.uf,
  SUM(f.perda_esperada_soma) AS perda_esperada_total
FROM puc.gold.fato_risco_cliente_mes f
JOIN puc.gold.dim_localizacao l
  ON l.sk_localizacao = f.sk_localizacao
JOIN ult_mes u
  ON f.dt_mes = u.dt_mes
GROUP BY l.uf
ORDER BY perda_esperada_total DESC;


In [0]:
WITH ult_mes AS (
  SELECT max(dt_mes) AS dt_mes
  FROM puc.gold.fato_risco_cliente_mes
),
agg AS (
  SELECT
    l.uf,
    SUM(f.valor_recuperado_soma) AS valor_recuperado,
    SUM(f.perda_esperada_soma) AS perda_esperada
  FROM puc.gold.fato_risco_cliente_mes f
  JOIN puc.gold.dim_localizacao l
    ON l.sk_localizacao = f.sk_localizacao
  JOIN ult_mes u
    ON f.dt_mes = u.dt_mes
  GROUP BY l.uf
)
SELECT
  uf,
  valor_recuperado,
  perda_esperada,
  valor_recuperado / NULLIF(perda_esperada, 0) AS taxa_recuperacao_proxy
FROM agg
ORDER BY taxa_recuperacao_proxy DESC;


In [0]:
WITH ult_mes AS (
  SELECT max(dt_mes) AS dt_fim
  FROM puc.gold.fato_risco_cliente_mes
),
base AS (
  SELECT *
  FROM puc.gold.fato_risco_cliente_mes
  WHERE dt_mes >= add_months((SELECT dt_fim FROM ult_mes), -11)
)
SELECT
  date_trunc('month', b.dt_mes) AS dt_mes,
  l.uf,
  SUM(b.ead_soma) AS ead_total
FROM base b
JOIN puc.gold.dim_localizacao l
  ON l.sk_localizacao = b.sk_localizacao
GROUP BY date_trunc('month', b.dt_mes), l.uf
ORDER BY dt_mes, ead_total DESC;


In [0]:
WITH meses AS (
  SELECT
    max(dt_mes) AS dt_atual,
    add_months(max(dt_mes), -1) AS dt_anterior
  FROM puc.gold.fato_risco_cliente_mes
),
agg AS (
  SELECT
    l.uf,
    f.dt_mes,
    SUM(f.ead_soma) AS ead_total
  FROM puc.gold.fato_risco_cliente_mes f
  JOIN puc.gold.dim_localizacao l
    ON l.sk_localizacao = f.sk_localizacao
  JOIN meses m
    ON f.dt_mes IN (m.dt_atual, m.dt_anterior)
  GROUP BY l.uf, f.dt_mes
)
SELECT
  uf,
  MAX(CASE WHEN dt_mes = (SELECT dt_atual FROM meses) THEN ead_total END) AS ead_atual,
  MAX(CASE WHEN dt_mes = (SELECT dt_anterior FROM meses) THEN ead_total END) AS ead_anterior,
  (MAX(CASE WHEN dt_mes = (SELECT dt_atual FROM meses) THEN ead_total END)
   - MAX(CASE WHEN dt_mes = (SELECT dt_anterior FROM meses) THEN ead_total END)) AS delta_ead,
  (MAX(CASE WHEN dt_mes = (SELECT dt_atual FROM meses) THEN ead_total END)
   / NULLIF(MAX(CASE WHEN dt_mes = (SELECT dt_anterior FROM meses) THEN ead_total END), 0) - 1) AS pct_var_ead
FROM agg
GROUP BY uf
ORDER BY pct_var_ead DESC;


In [0]:
WITH ult_mes AS (
  SELECT max(dt_mes) AS dt_mes
  FROM puc.gold.fato_risco_cliente_mes
),
uf_ead AS (
  SELECT
    l.uf,
    SUM(f.ead_soma) AS ead_total
  FROM puc.gold.fato_risco_cliente_mes f
  JOIN puc.gold.dim_localizacao l
    ON l.sk_localizacao = f.sk_localizacao
  JOIN ult_mes u
    ON f.dt_mes = u.dt_mes
  GROUP BY l.uf
),
ord AS (
  SELECT
    uf,
    ead_total,
    SUM(ead_total) OVER () AS ead_geral,
    SUM(ead_total) OVER (ORDER BY ead_total DESC) AS ead_acumulado
  FROM uf_ead
)
SELECT
  uf,
  ead_total,
  ead_acumulado / ead_geral AS pct_acumulado
FROM ord
WHERE (ead_acumulado / ead_geral) <= 0.80
ORDER BY ead_total DESC;


In [0]:
WITH ult_12_meses AS (
  SELECT dt_mes
  FROM (
    SELECT DISTINCT dt_mes
    FROM puc.gold.fato_risco_cliente_mes
  )
  ORDER BY dt_mes DESC
  LIMIT 12
),
uf_ead AS (
  SELECT
    f.dt_mes,
    l.uf,
    SUM(f.ead_soma) AS ead_total
  FROM puc.gold.fato_risco_cliente_mes f
  JOIN puc.gold.dim_localizacao l
    ON l.sk_localizacao = f.sk_localizacao
  JOIN ult_12_meses u
    ON f.dt_mes = u.dt_mes
  GROUP BY f.dt_mes, l.uf
),
ord AS (
  SELECT
    dt_mes,
    uf,
    ead_total,
    SUM(ead_total) OVER (PARTITION BY dt_mes) AS ead_geral,
    SUM(ead_total) OVER (
      PARTITION BY dt_mes
      ORDER BY ead_total DESC
      ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) AS ead_acumulado
  FROM uf_ead
)
SELECT
  dt_mes,
  uf,
  ead_total,
  ead_acumulado / ead_geral AS pct_acumulado
FROM ord
WHERE (ead_acumulado / ead_geral) <= 0.80
ORDER BY dt_mes DESC, ead_total DESC;


## Cliente

In [0]:
WITH ult_mes AS (
  SELECT max(dt_mes) AS dt_mes
  FROM puc.gold.fato_risco_cliente_mes
)
SELECT
  CASE
    WHEN c.tempo_conta_anos < 1 THEN '<1 ano'
    WHEN c.tempo_conta_anos BETWEEN 1 AND 3 THEN '1-3 anos'
    WHEN c.tempo_conta_anos BETWEEN 4 AND 7 THEN '4-7 anos'
    WHEN c.tempo_conta_anos BETWEEN 8 AND 15 THEN '8-15 anos'
    ELSE '15+ anos'
  END AS faixa_tempo_conta,
  SUM(f.ead_soma) AS ead_total,
  SUM(f.pd_true_media * f.ead_soma) / NULLIF(SUM(f.ead_soma), 0) AS pd_true_pond
FROM puc.gold.fato_risco_cliente_mes f
JOIN puc.gold.dim_cliente c ON c.sk_cliente = f.sk_cliente
JOIN ult_mes u ON f.dt_mes = u.dt_mes
GROUP BY faixa_tempo_conta
ORDER BY ead_total DESC;


In [0]:
WITH ult_mes AS (
  SELECT max(dt_mes) AS dt_mes
  FROM puc.gold.fato_risco_cliente_mes
)
SELECT
  CASE
    WHEN c.tempo_emprego_anos < 1 THEN '<1 ano'
    WHEN c.tempo_emprego_anos BETWEEN 1 AND 3 THEN '1-3 anos'
    WHEN c.tempo_emprego_anos BETWEEN 4 AND 7 THEN '4-7 anos'
    WHEN c.tempo_emprego_anos BETWEEN 8 AND 15 THEN '8-15 anos'
    ELSE '15+ anos'
  END AS faixa_tempo_emprego,
  SUM(f.ead_soma) AS ead_total,
  SUM(f.pd_true_media * f.ead_soma) / NULLIF(SUM(f.ead_soma), 0) AS pd_true_pond
FROM puc.gold.fato_risco_cliente_mes f
JOIN puc.gold.dim_cliente c ON c.sk_cliente = f.sk_cliente
JOIN ult_mes u ON f.dt_mes = u.dt_mes
GROUP BY faixa_tempo_emprego
ORDER BY ead_total DESC;


In [0]:
WITH ult_mes AS (
  SELECT max(dt_mes) AS dt_mes
  FROM puc.gold.fato_risco_cliente_mes
)
SELECT
  c.setor,
  COUNT(DISTINCT c.sk_cliente) AS qtd_clientes,
  SUM(f.ead_soma) AS ead_total,
  SUM(f.pd_model_media * f.ead_soma) / NULLIF(SUM(f.ead_soma), 0) AS pd_model_pond,
  SUM(f.pd_true_media  * f.ead_soma) / NULLIF(SUM(f.ead_soma), 0) AS pd_true_pond
FROM puc.gold.fato_risco_cliente_mes f
JOIN puc.gold.dim_cliente c ON c.sk_cliente = f.sk_cliente
JOIN ult_mes u ON f.dt_mes = u.dt_mes
GROUP BY c.setor
ORDER BY pd_true_pond DESC;


In [0]:
WITH ult_mes AS (
  SELECT max(dt_mes) AS dt_mes
  FROM puc.gold.fato_risco_cliente_mes
)
SELECT
  c.escolaridade,
  COUNT(*) AS qtd_clientes,
  SUM(f.ead_soma) AS ead_total,
  SUM(f.pd_true_media * f.ead_soma) / NULLIF(SUM(f.ead_soma), 0) AS pd_true_pond,
  SUM(f.perda_esperada_soma) AS perda_esperada_total
FROM puc.gold.fato_risco_cliente_mes f
JOIN puc.gold.dim_cliente c ON c.sk_cliente = f.sk_cliente
JOIN ult_mes u ON f.dt_mes = u.dt_mes
GROUP BY c.escolaridade
ORDER BY ead_total DESC;


In [0]:
WITH ult_mes AS (
  SELECT max(dt_mes) AS dt_mes
  FROM puc.gold.fato_risco_cliente_mes
),
base AS (
  SELECT
    c.idade,
    f.ead_soma,
    f.perda_esperada_soma,
    f.pd_model_media,
    f.pd_true_media
  FROM puc.gold.fato_risco_cliente_mes f
  JOIN puc.gold.dim_cliente c ON c.sk_cliente = f.sk_cliente
  JOIN ult_mes u ON f.dt_mes = u.dt_mes
),
faixas AS (
  SELECT
    CASE
      WHEN idade < 18 THEN '<18'
      WHEN idade BETWEEN 18 AND 25 THEN '18-25'
      WHEN idade BETWEEN 26 AND 35 THEN '26-35'
      WHEN idade BETWEEN 36 AND 45 THEN '36-45'
      WHEN idade BETWEEN 46 AND 55 THEN '46-55'
      WHEN idade BETWEEN 56 AND 65 THEN '56-65'
      ELSE '66+'
    END AS faixa_idade,
    *
  FROM base
)
SELECT
  faixa_idade,
  COUNT(*) AS qtd_registros,
  SUM(ead_soma) AS ead_total,
  SUM(perda_esperada_soma) AS perda_esperada_total,
  SUM(pd_model_media * ead_soma) / NULLIF(SUM(ead_soma), 0) AS pd_model_pond,
  SUM(pd_true_media  * ead_soma) / NULLIF(SUM(ead_soma), 0) AS pd_true_pond
FROM faixas
GROUP BY faixa_idade
ORDER BY ead_total DESC;


In [0]:
WITH ult_mes AS (
  SELECT max(dt_mes) AS dt_mes
  FROM puc.gold.fato_risco_cliente_mes
)
SELECT
  c.idade,
  AVG(f.inadimplencia_media) AS inadimplencia_media
FROM puc.gold.fato_risco_cliente_mes f
JOIN puc.gold.dim_cliente c ON c.sk_cliente = f.sk_cliente
JOIN ult_mes u ON f.dt_mes = u.dt_mes
GROUP BY c.idade
ORDER BY c.idade;


In [0]:
WITH ult_mes AS (
  SELECT MAX(dt_mes) AS dt_mes
  FROM puc.gold.fato_risco_cliente_mes
)
SELECT
  c.escolaridade,
  COUNT(DISTINCT c.sk_cliente) AS qtd_clientes,
  SUM(f.ead_soma) AS ead_total,
  SUM(f.pd_model_media * f.ead_soma) / NULLIF(SUM(f.ead_soma), 0) AS pd_model_pond,
  SUM(f.pd_true_media  * f.ead_soma) / NULLIF(SUM(f.ead_soma), 0) AS pd_true_pond
FROM puc.gold.fato_risco_cliente_mes f
JOIN puc.gold.dim_cliente c
  ON c.sk_cliente = f.sk_cliente
JOIN ult_mes u
  ON f.dt_mes = u.dt_mes
GROUP BY c.escolaridade
ORDER BY pd_true_pond DESC;
